In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv
/kaggle/input/icr-identify-age-related-conditions/greeks.csv
/kaggle/input/icr-identify-age-related-conditions/train.csv
/kaggle/input/icr-identify-age-related-conditions/test.csv


# 1. Import the Libraries

In [2]:
import torch
from torch import tensor
from fastai.data.transforms import RandomSplitter
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder, StandardScaler
import lightgbm as lgb
from sklearn.metrics import log_loss, confusion_matrix, roc_curve, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 2. Load the Datasets

In [4]:
train_df = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/train.csv")
test_df = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/test.csv")
greeks_df = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/greeks.csv")
sample_submission_df = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv")

# 3. View the Dataset

In [5]:
train_df.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,22.5984,175.638726,152.707705,823.928241,257.432377,47.223358,0.563481,23.387600,4.851915,0.023482,1.050225,0.069225,13.784111,1.302012,36.205956,69.08340,295.570575,0.23868,0.284232,89.245560,84.31664,29.657104,5.310690,1.74307,23.187704,7.294176,1.987283,1433.166750,0.949104,B,30.879420,78.526968,3.828384,13.394640,10.265073,9028.291921,3.583450,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,19.4205,155.868030,14.754720,51.216883,257.432377,30.284345,0.484710,50.628208,6.085041,0.031442,1.113875,1.117800,28.310953,1.357182,37.476568,70.79836,178.553100,0.23868,0.363489,110.581815,75.74548,37.532000,0.005518,1.74307,17.222328,4.926396,0.858603,1111.287150,0.003042,A,109.125159,95.415086,52.260480,17.175984,0.296850,6785.003474,10.358927,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,26.4825,128.988531,219.320160,482.141594,257.432377,32.563713,0.495852,85.955376,5.376488,0.036218,1.050225,0.700350,39.364743,1.009611,21.459644,70.81970,321.426625,0.23868,0.210441,120.056438,65.46984,28.053464,1.289739,1.74307,36.861352,7.813674,8.146651,1494.076488,0.377208,B,109.125159,78.526968,5.390628,224.207424,8.745201,8338.906181,11.626917,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,23.6577,237.282264,11.050410,661.518640,257.432377,15.201914,0.717882,88.159360,2.347652,0.029054,1.400300,0.636075,41.116960,0.722727,21.530392,47.27586,196.607985,0.23868,0.292431,139.824570,71.57120,24.354856,2.655345,1.74307,52.003884,7.386060,3.813326,15691.552180,0.614484,B,31.674357,78.526968,31.323372,59.301984,7.884336,10965.766040,14.852022,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,24.0108,324.546318,149.717165,6074.859475,257.432377,82.213495,0.536467,72.644264,30.537722,0.025472,1.050225,0.693150,31.724726,0.827550,34.415360,74.06532,200.178160,0.23868,0.207708,97.920120,52.83888,26.019912,1.144902,1.74307,9.064856,7.350720,3.490846,1403.656300,0.164268,B,109.125159,91.994825,51.141336,29.102640,4.274640,16198.049590,13.666727,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


# 4. Desciptive Statistics of train dataset

In [6]:
train_df.describe()

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
count,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,557.000000,617.000000,617.000000,615.000000,614.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,616.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,557.000000,617.000000,617.000000,616.000000,617.000000,617.000000,617.000000,616.000000,617.000000,615.000000,617.000000,617.000000,617.000000,617.000000,617.000000,616.000000,617.000000
mean,0.477149,3502.013221,118.624513,38.968552,10.128242,5.545576,0.060320,10.566447,8.053012,5350.388655,21.419492,231.322223,98.328737,1218.133238,550.632525,77.104151,0.688801,90.251735,11.241064,0.030615,1.403761,0.742262,36.917590,1.383792,27.165653,51.128326,401.901299,0.633884,0.367002,146.972099,94.795377,26.370568,1.802900,1.924830,26.388989,9.072700,3.064778,1731.248215,0.305107,69.582596,105.060712,69.117005,71.341526,6.930086,10306.810737,10.111079,5.433199,3.533905,0.421501,20.724856,131.714987,14679.595398,31.489716,50.584437,8.530961,0.175041
std,0.468388,2300.322717,127.838950,69.728226,10.518877,2.551696,0.416817,4.350645,65.166943,3021.326641,3.478278,183.992505,96.479371,7575.293707,2076.371275,159.049302,0.263994,51.585130,13.571133,0.014808,1.922210,0.281195,17.266347,0.538717,14.645993,21.210888,317.745623,1.912384,0.112989,86.084419,28.243187,8.038825,9.034721,1.484555,18.116679,6.200281,2.058344,1790.227476,1.847499,38.555707,68.445620,390.187057,165.551545,64.754262,11331.294051,2.934025,11.496257,50.181948,1.305365,9.991907,144.181524,19352.959387,9.864239,36.266251,10.327010,0.380310
min,0.081187,192.593280,85.200147,3.177522,8.138688,0.699861,0.025578,3.396778,1.229900,1693.624320,9.886800,72.948951,1.331155,51.216883,257.432377,12.499760,0.176874,23.387600,0.510888,0.003184,1.050225,0.069225,13.784111,0.137925,7.030640,6.906400,35.998895,0.238680,0.040995,60.232470,10.345600,6.339496,0.005518,1.743070,0.804068,4.926396,0.286201,185.594100,0.003042,5.394675,78.526968,3.828384,7.534128,0.296850,1563.136688,3.583450,0.173229,0.497060,0.067730,4.102182,72.611063,13.038894,9.432735,0.897628,0.001129,0.000000
25%,0.252107,2197.345480,85.200147,12.270314,8.138688,4.128294,0.025578,8.129580,1.229900,4155.702870,19.420500,156.847239,27.834425,424.990642,257.432377,23.317567,0.563688,64.724192,5.066306,0.023482,1.050225,0.589575,29.782467,1.070298,7.030640,37.942520,188.815690,0.238680,0.295164,102.703553,78.232240,20.888264,0.005518,1.743070,14.715792,5.965392,1.648679,1111.160625,0.003042,30.927468,78.526968,4.324656,25.815384,0.296850,5164.666260,8.523098,0.173229,0.497060,0.067730,14.036718,72.611063,2798.992584,25.034888,23.011684,0.124392,0.000000
50%,0.354659,3120.318960,85.200147,20.533110,8.138688,5.031912,0.025578,10.461320,1.229900,4997.960730,21.186000,193.908816,61.642115,627.417402,257.432377,42.554330,0.658715,79.819104,9.123000,0.027860,1.050225,0.730800,34.835130,1.351665,36.019104,49.180940,307.509595,0.238680,0.358023,130.050630,96.264960,25.248800,0.251741,1.743070,21.642456,8.149404,2.616119,1493.817413,0.085176,71.949306,78.526968,22.641144,36.394008,1.870155,7345.143424,9.945452,3.028141,1.131000,0.250601,18.771436,72.611063,7838.273610,30.608946,41.007968,0.337827,0.000000
75%,0.559763,4361.637390,113.739540,39.139886,8.138688,6.431634,0.036845,12.969516,5.081244,6035.885700,23.657700,247.803462,134.009015,975.649259,257.432377,77.310097,0.772206,99.813520,13.565901,0.034427,1.228445,0.859350,40.529401,1.660617,37.935832,61.408760,507.896200,0.238680,0.426348,165.836955,110.640680,30.544224,1.058690,1.743070,34.058344,10.503048,3.910070,1905.701475,0.237276,109.125159,112.766654,49.085352,56.714448,4.88021

# 5. Encoding

In [7]:
def encode(dataframe):
    le = LabelEncoder()
    obj = list(dataframe.loc[:, dataframe.dtypes == 'object'].columns)
    for i in obj:
        if i not in ['id', 'Epsilon']:
            dataframe[i] = le.fit_transform(dataframe[i])
    return dataframe

In [8]:
df = encode(train_df)
test_df = encode(test_df)

In [9]:
df.columns

Index(['Id', 'AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN',
       'BP', 'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS',
       'CU', 'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY',
       'EB', 'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI',
       'FL', 'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL', 'Class'],
      dtype='object')

# 6. Separate into Dependent and Independent 

In [10]:
indep_cols = ['Id', 'AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN',
       'BP', 'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS',
       'CU', 'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY',
       'EB', 'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI',
       'FL', 'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL']

dep_cols = ['Class']

# 7. Simple Imputer

In [11]:
imputer = SimpleImputer(strategy = 'mean')
df[indep_cols] = imputer.fit_transform(df[indep_cols])
test_df[indep_cols] = imputer.fit_transform(test_df[indep_cols])

In [12]:
list(imputer.statistics_)

[2.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

# 8. MinMax Scaler 

In [13]:
scaler = MinMaxScaler()

In [14]:
scaler.fit(df[indep_cols])

MinMaxScaler()

**Minimum**

In [15]:
print('Minimum:')
list(scaler.data_min_)

Minimum:


[0.0,
 0.081187,
 192.59328,
 85.200147,
 3.177522,
 8.138688,
 0.699861,
 0.025578,
 3.396778,
 1.2299,
 1693.62432,
 9.8868,
 72.948951,
 1.331155,
 51.216883,
 257.432377,
 12.49976,
 0.17687412,
 23.3876,
 0.510888,
 0.003184,
 1.050225,
 0.069225,
 13.784111,
 0.137925,
 7.03064,
 6.9064,
 35.998895,
 0.23868,
 0.040995,
 60.23247,
 10.3456,
 6.339496,
 0.0055176,
 1.74307,
 0.804068,
 4.926396,
 0.286201,
 185.5941,
 0.003042,
 0.0,
 5.394675,
 78.526968,
 3.828384,
 7.534128,
 0.29685,
 1563.136688,
 3.58345,
 0.173229,
 0.49706,
 0.06773,
 4.102182,
 72.611063,
 13.038894,
 9.432735,
 0.897628,
 0.001129278]

**Maximum**

In [16]:
print('Maximum:')
list(scaler.data_max_)

Maximum:


[616.0,
 6.161666,
 28688.18766,
 1910.123198,
 630.51823,
 178.943634,
 38.27088,
 10.315851,
 38.971568,
 1463.693448,
 53060.59924,
 29.3073,
 2447.81055,
 344.644105,
 179250.2529,
 50092.4593,
 2271.436167,
 4.1030316,
 633.534408,
 200.967526,
 0.224074,
 31.6881525,
 3.039675,
 267.9428235,
 4.9515075,
 64.521624,
 210.33092,
 2103.40519,
 37.895013,
 1.060404,
 1049.168078,
 326.2362,
 62.808096,
 161.355315,
 25.19293,
 152.355164,
 94.95858,
 18.324926,
 30243.75878,
 42.569748,
 1.0,
 109.125159,
 1063.594578,
 6501.26448,
 3030.655824,
 1578.654237,
 143224.6823,
 35.851039,
 137.9327388,
 1244.22702,
 31.365763,
 135.781294,
 1497.351958,
 143790.0712,
 81.210825,
 191.194764,
 21.978]

In [17]:
df[indep_cols] = scaler.transform(df[indep_cols])

In [18]:
df[indep_cols].describe()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
count,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000
mean,0.500000,0.065120,0.116138,0.018315,0.057052,0.011648,0.128975,0.003376,0.201538,0.004665,0.071189,0.593841,0.066687,0.282534,0.006512,0.005883,0.028599,0.130389,0.109587,0.053529,0.124183,0.011539,0.226577,0.091020,0.258823,0.350229,0.217387,0.176986,0.010495,0.319800,0.087710,0.267339,0.354729,0.011140,0.007751,0.168820,0.046054,0.154034,0.051422,0.007096,0.640194,0.618795,0.026936,0.010048,0.021106,0.004203,0.061722,0.202297,0.038182,0.002442,0.011303,0.126236,0.041484,0.102009,0.307294,0.261101,0.388128
std,0.289378,0.077031,0.080726,0.070052,0.111149,0.061584,0.067917,0.040506,0.122296,0.044560,0.058818,0.179103,0.077475,0.266988,0.042273,0.041665,0.070295,0.067076,0.084545,0.067701,0.067039,0.062740,0.094664,0.067935,0.111916,0.254753,0.104269,0.153693,0.050785,0.110838,0.087048,0.089408,0.142359,0.055949,0.063308,0.119542,0.068867,0.114107,0.059559,0.043402,0.480333,0.353126,0.069483,0.060052,0.054717,0.041026,0.079988,0.090928,0.083384,0.040348,0.041640,0.075881,0.101198,0.134604,0.137427,0.190577,0.469522
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.250000,0.028110,0.070353,0.000000,0.014494,0.000000,0.091252,0.000000,0.133038,0.000000,0.047931,0.490909,0.035328,0.083785,0.002086,0.000000,0.004789,0.098681,0.067749,0.022725,0.091892,0.000000,0.175175,0.062946,0.193696,0.000000,0.152568,0.073917,0.000000,0.249330,0.042946,0.214906,0.257644,0.000000,0.000000,0.091796,0.011540,0.075531,0.030793,0.000000,0.000000,0.269686,0.000000,0.000076,0.006056,0.000000,0.025423,0.153084,0.000000,0.000000,0.000000,0.075445,0.000000,0.019377,0.217367,0.116208,0.005610
50%,0.500000,0.044975,0.102743,0.000000,0.027665,0.000000,0.115303,0.000000,0.198583,0.000000,0.064328,0.581818,0.050933,0.209618,0.003215,0.000000,0.013358,0.123009,0.092488,0.042962,0.111712,0.000000,0.222719,0.082826,0.252149,0.504226,0.207814,0.131329,0.000000,0.310992,0.070599,0.271991,0.334864,0.001547,0.000000,0.137501,0.035798,0.129162,0.043523,0.001930,1.000000,0.618795,0.000000,0.002895,0.009566,0.000997,0.040816,0.197164,0.020787,0.000510,0.006059,0.111402,0.000000,0.054426,0.295023,0.210777,0.015393
75%,0.750000,0.078707,0.146305,0.015639,0.057325,0.000000,0.152558,0.001095,0.269088,0.002633,0.084534,0.709091,0.073627,0.354078,0.005159,0.000000,0.028670,0.151557,0.125258,0.065126,0.141441,0.005817,0.265995,0.105231,0.316332,0.537566,0.267924,0.228256,0.000000,0.378016,0.106786,0.317499,0.428640,0.006591,0.000000,0.219426,0.061941,0.200894,0.057226,0.005503,1.000000,1.000000,0.034759,0.006965,0.016279,0.002904,0.064130,0.245857,0.044019,0.000816,0.014932,0.163323,0.038590,0.132307,0.382167,0.352260,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000

# 9. Train the model

In [19]:
X = df[indep_cols]
y = df[dep_cols]

In [20]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []

**Logistic Regression**

In [21]:
for train_idx, val_idx in skf.split(X, y):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_valid, y_valid = X.iloc[val_idx], y.iloc[val_idx]
    model = LogisticRegression(solver='liblinear')    
    model.fit(X_train, y_train)
    val_preds = model.predict_proba(X_valid)
    val_score = log_loss(y_valid, val_preds)
    scores.append(val_score)

print(f'Log-loss scores: {scores}')
print('*' * 45)
print(f'Mean Log-loss: {np.mean(scores)}')

Log-loss scores: [0.32225114643241054, 0.3403899503127844, 0.3592379987382104, 0.34510276634634535, 0.30361286139047855]
*********************************************
Mean Log-loss: 0.33411894464404585


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

***Checking the weights and biases of the trained mode***

In [22]:
print(model.coef_.tolist())

[[-0.048052289727862726, 0.8901303031883299, 1.3150279115943055, 0.13174306487470439, 0.2622248197824984, 0.0390137362811528, -0.26887734576414224, 0.3189733468464598, -0.20667907391503582, 0.5075642847440733, 0.4732638629119724, 1.588431634183748, 0.6643016265569007, 1.6813665369969006, 0.534784482343446, 0.1569225386396318, -0.18079557638749433, -0.71948639212963, 1.323381673155981, 0.5661967044583167, -0.6725976239897116, 0.44640002874247514, -2.104515356021582, -0.4106919581294929, -0.476265241357835, -0.33458320371675826, -0.9935583565069934, -0.6994955837121527, -0.2514334312777739, -1.5392087395159657, 1.0610628850468167, -1.094798666372673, -0.9686861797798433, 1.423025118877475, 0.4905439829928888, 0.8913732117099116, 0.4053340712554371, -1.05398978966617, -0.40837875387742967, 0.5336512972480261, -0.6399629894331432, 0.19323320859572463, -0.47744452211118044, -0.25097807252395554, 0.02380150507745937, 0.22365378392300575, 1.44986245473914, -0.966989469933129, 1.10128027252566

In [23]:
print(model.intercept_)

[-0.57293004]


# 10. Sample Submission File

In [24]:
sample_submission_df.head()

,Id,class_0,class_1
0,00eed32682bb,0.5,0.5
1,010ebe33f668,0.5,0.5
2,02fa521e1838,0.5,0.5
3,040e15f562a2,0.5,0.5
4,046e85c7cc7f,0.5,0.5


# 11. Prediction of Final Submission File

In [25]:
prediction = model.predict_proba(test_df[indep_cols])

In [26]:
sample_submission_df[['class_0', 'class_1']] = prediction

In [27]:
sample_submission_df.head()

,Id,class_0,class_1
0,00eed32682bb,0.639439,0.360561
1,010ebe33f668,0.650442,0.349558
2,02fa521e1838,0.661287,0.338713
3,040e15f562a2,0.671965,0.328035
4,046e85c7cc7f,0.682468,0.317532


In [28]:
sample_submission_df.to_csv('submission.csv', index=False)